# Assignment 2 Stochastic Simulation - Discrete Event Simulation 

In this assignment we will use the following notation:
- $\lambda$ - the arrival rate into the system.
- $\mu$ - the capacity of eaach of $\textit{n}$ equal servers.
- $\rho$ - the system load. In a single server system, it will be 
$\rho = \frac{\lambda}{\mu}$. In a multi-server system (one queue with $\textit{n}$ equal servers, each with capacity $\mu$), it will be $\rho = \frac{\lambda}{n \mu}$

The mean waiting time for an M/M/1 queuing system is defined by:

$$
E(W) = E(s) - \frac{1}{\mu} = \frac{\frac{\rho}{\mu}}{1 - \rho}
$$


The mean waiting time for an M/M/n queuing system is defined by:

$$
E(W) = \Pi_W \cdot \frac{1}{1 - \rho} \cdot \frac{1}{n \mu}
$$

where \( n \) is defined as the number of servers that are available.

In the case of multiple servers, the probability that all servers are full has to be taken into account. 
This is defined as:

$$
\Pi_W = \rho_n + \rho_{n+1} + \rho_{n+2} + \cdots = \frac{\rho_{c}}{1 - \rho}
$$


In [54]:
import numpy
import simpy
from random import seed
from scipy.stats import t
from scipy.stats import ttest_ind
import random
import statistics

In [52]:
number_of_servers = [1,2,4]

def simulate_mmn(new_customers, mean_interarrival, mean_service_time, n, random_seed):
    random.seed(random_seed)

    waitingTimes = []
    serviceTimes = []
    interarrivalTimes = []

    def generator(env, number, interval, server):  # customer generator with interarrival times.
        """generator generates customers randomly"""
        for i in range(number):
            c = customer(env, 'Customer%02d' % i, server, service_time=random.expovariate(1.0/mean_service_time))
            env.process(c)
            t = random.expovariate(1.0/ interval)
            interarrivalTimes.append(t)
            yield env.timeout(t)  # adds time to the counter, does not delete from the memory

    def customer(env, name, server, service_time):
        # customer arrives to the system, waits and leaves
        arrive = env.now
        #print('%7.4f : Arrival time of %s' % (arrive, name))
        with server.request() as req:
            results = yield req | env.timeout(arrive)
            
            if req in results:
                servertime = service_time
                yield env.timeout(servertime)
                serviceTimes.append(servertime)
                #print('%7.4f Departure Time of %s' % (env.now, name))
                #print('%7.4f Time Spent in the system of %s' % (env.now - arrive, name))
            else:
                waiting_time = env.now - arrive
                waitingTimes.append(waiting_time)
                #print('%6.3f Waiting time of %s' % (waiting_time, name))
    
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=n)  # capacity changes the number of generators in the system.
    env.process(generator(env, new_customers, mean_interarrival, server))
    env.run()
    #interarrivalTimes.append(interarrival)

    average_interarrival = statistics.mean(interarrivalTimes)
    average_waitingTime = statistics.mean(waitingTimes) if waitingTimes else 0
    average_serviceTime = statistics.mean(serviceTimes)
    return {
        "servers": n,
        "average_interarrival": average_interarrival,
        "average_waitingTime": average_waitingTime,
        "average_serviceTime": average_serviceTime
    }

def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = numpy.mean(data)
    std_err = numpy.std(data, ddof=1) / numpy.sqrt(n)
    t_value = t.ppf((1+ confidence)/2, df=n-1)
    return mean, mean - t_value * std_err, mean + t_value * std_err  

system_load = 0.95
new_customers = 100
mean_service_time = 4
simulations = 25
servers = [1,2,4]

random.seed(42)

service_rate = 1/mean_service_time
mean_interarrival_by_n = {n: 1 / (n*system_load * service_rate) for n in servers}

results = {1: [], 2: [], 4: []}

for n in servers:
    mean_interarrival = mean_interarrival_by_n[n]
    for _ in range(simulations):
        random_seed = random.randint(0,10000)
        result = simulate_mmn(new_customers, mean_interarrival, mean_service_time, n, random.randint(0, 10000))
        if result['average_waitingTime'] > 0:
            results[n].append(result['average_waitingTime'])


print("\nSimulation results for n = 1,2 and 4:")
# Output results with confidence intervals
print("\nSimulation results for n=1, 2, and 4 with Confidence Intervals:")
for n in results:
    mean, ci_lower, ci_upper = confidence_interval(results[n]) 
    print(
        f"Servers: {n}, "
        f"Mean Waiting Time: {mean:.4f}, "
        f"95% CI: ({ci_lower:.4f}, {ci_upper:.4f})"
    )




Simulation results for n = 1,2 and 4:

Simulation results for n=1, 2, and 4 with Confidence Intervals:
Servers: 1, Mean Waiting Time: 9.7907, 95% CI: (4.4324, 15.1489)
Servers: 2, Mean Waiting Time: 3.7731, 95% CI: (1.3225, 6.2237)
Servers: 4, Mean Waiting Time: 4.6815, 95% CI: (nan, nan)


In [53]:
t_stat, p_value = ttest_ind(results[1], results[2], equal_var=False)
print(f"T-statistic: {t_stat:.4f}, P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Result is statistically significant and we can reject H0")
else:
    print("Result is not statistically significant and we fail to reject H0")

T-statistic: 2.1858, P-value: 0.0397
Result is statistically significant and we can reject H0
